In [ ]:
import pandas as pd
import cryptpandas as crp
from slack_sdk import WebClient
import re
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2 import service_account
import io
import time
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os

# Slack API credentials
slack_token = ""
channel_id = "C080P6M4DKL"
expected_user_email = "------@gmail.com"

# Google Drive API credentials
SERVICE_ACCOUNT_FILE = 'direct-volt-441912-r9-6b6d82e81e93.json'  # Replace with the path to your JSON key file
SCOPES = ['https://www.googleapis.com/auth/drive']
FOLDER_ID = '1ElVOO_4Plr24xEOmdqsINmIRM_y4M3_n'


# Initialize the Slack client
client = WebClient(token=slack_token)

def fetch_recent_messages(channel_id, limit=30):
    """
    Fetch recent messages from a Slack channel.
    """
    try:
        response = client.conversations_history(channel=channel_id, limit=limit)
        messages = response.get('messages', [])
        return messages
    except Exception as e:
        print(f"Error fetching messages: {e}")
        return []

def get_user_email(user_id):
    """
    Fetch the email of a user based on their Slack user ID.
    """
    try:
        response = client.users_info(user=user_id)
        email = response['user']['profile'].get('email', '')
        return email
    except Exception as e:
        print(f"Error fetching user info: {e}")
        return None

def extract_data_file_and_passcode(messages):
    """
    Extract the data file name and passcode from the messages.
    """
    for message in messages:
        text = message.get('text', '')
        user_id = message.get('user', '')
        # Verify the user's email
        user_email = get_user_email(user_id)
        if user_email == expected_user_email:
            match = re.search(
                r"Data has just been released '(.+?)' the passcode is '(.+?)'\.",
                text
            )
            if match:
                file_name = match.group(1)
                passcode = match.group(2)
                return file_name, passcode
    return None, None

def authenticate_google_drive():
    """
    Authenticate and initialize the Google Drive API client.
    """
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)
    return service

def find_and_download_file(service, file_name):
    """
    Find a file by name in the specified folder and download it.
    """
    query = f"'{FOLDER_ID}' in parents and name='{file_name}'"
    results = service.files().list(q=query, spaces='drive').execute()
    files = results.get('files', [])

    if not files:
        print(f"File '{file_name}' not found in the folder.")
        return False

    file_id = files[0]['id']
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}% complete.")
    
    print(f"File '{file_name}' downloaded successfully.")
    return True

def decrypt_file(file_name, passcode):
    """
    Decrypt the downloaded file using the provided passcode.
    """
    try:
        decrypted_df = crp.read_encrypted(path=file_name, password=passcode)
        decrypted_df.to_csv('latest_data.csv', index=False)
        print("Decrypted DataFrame:")
        print(decrypted_df)
        return decrypted_df
    except Exception as e:
        print(f"Error decrypting file: {e}")
        return None

def process_data(decrypted_df):
    """
    Process the decrypted data and perform modeling and optimization.
    """
    df_filled = decrypted_df.ffill().bfill().fillna(decrypted_df.mean())
    # Replace infinite values with NaN
    df_filled.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Check for NaN values
    print("Nombre total de valeurs NaN après remplacement des infinis :", df_filled.isna().sum().sum())
    # Replace NaNs with column means
    df_filled.fillna(df_filled.mean(), inplace=True)
    print("Nombre total de valeurs NaN après traitement :", df_filled.isna().sum().sum())
    returns = df_filled

    # Data Preprocessing
    returns = returns.replace([np.inf, -np.inf], np.nan)
    print("Nombre total de valeurs NaN après remplacement des infinis :", returns.isna().sum().sum())
    returns = returns.dropna(axis=1)
    print("Nombre total de valeurs NaN après traitement :", returns.isna().sum().sum())
    print("Returns DataFrame shape after preprocessing:", returns.shape)

    # Scale the returns data
    scaler = StandardScaler()
    returns_scaled = pd.DataFrame(scaler.fit_transform(returns), columns=returns.columns, index=returns.index)
    print("Returns DataFrame after scaling:")
    print(returns_scaled.head())

    # Function to create lagged features
    def create_lagged_features(data, lag=1):
        df = data.copy()
        column_name = df.columns[0]
        for i in range(1, lag+1):
            df[f'Lag_{i}'] = df[column_name].shift(i)
        return df.dropna()
    
    lag = 5  # Number of lagged days to use as features
    predicted_returns = pd.DataFrame(index=returns_scaled.index, columns=returns_scaled.columns)
    
    for asset in returns_scaled.columns:
        print(f"Training model for {asset}...")
        data = returns_scaled[[asset]]
        data_lagged = create_lagged_features(data, lag)
        X = data_lagged.drop(asset, axis=1)
        y = data_lagged[asset]
    
        # Time series split
        tscv = TimeSeriesSplit(n_splits=5)
        predictions = pd.Series(index=y.index)
    
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train = y.iloc[train_index]
    
            # XGBoost regressor
            model = XGBRegressor(objective='reg:squarederror', n_estimators=100)
            model.fit(X_train, y_train)
    
            # Predict
            y_pred = model.predict(X_test)
            predictions.iloc[test_index] = y_pred
    
        predicted_returns[asset] = predictions
    
    # Drop any rows with NaN predictions
    predicted_returns = predicted_returns.dropna()
    
    # Clip predicted returns
    predicted_returns = predicted_returns.clip(lower=-3, upper=3)
    
    # Re-scale predicted returns back to original scale
    predicted_returns_unscaled = pd.DataFrame(
        scaler.inverse_transform(predicted_returns),
        columns=predicted_returns.columns,
        index=predicted_returns.index
    )
    
    # Calculate expected returns from the predictions
    expected_returns = predicted_returns_unscaled.mean()
    print("\nExpected Returns per Asset:")
    print(expected_returns)
    
    # Calculate the covariance matrix from historical data (scaled returns)
    cov_matrix = returns_scaled.loc[predicted_returns.index].cov()
    
    # Optimization functions
    def portfolio_performance(weights, expected_returns, cov_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return portfolio_return, portfolio_volatility
    
    def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
        p_return, p_volatility = portfolio_performance(weights, expected_returns, cov_matrix)
        sharpe_ratio = (p_return - risk_free_rate) / p_volatility
        return -sharpe_ratio  # Negative because we minimize
    
    # Constraints: Sum of weights = 1, weights between 0% and 10%
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
    bounds = tuple((0, 0.1) for _ in range(len(expected_returns)))
    
    # Initial guess
    num_assets = len(expected_returns)
    initial_weights = num_assets * [1. / num_assets,]
    
    # Optimization
    options = {'disp': False}
    result = minimize(neg_sharpe_ratio, initial_weights, args=(expected_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints, options=options)
    
    optimal_weights = result.x
    p_return, p_volatility = portfolio_performance(optimal_weights, expected_returns, cov_matrix)
    sharpe_ratio = (p_return - 0.01) / p_volatility
    
    # Display results
    print("\nOptimal portfolio allocation:")
    for i, weight in enumerate(optimal_weights):
        if weight > 0:
            print(f"  {returns.columns[i]}: {weight:.16f}")
    print(f"Expected portfolio return: {p_return:.2%}")
    print(f"Portfolio volatility: {p_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    
    return returns.columns, optimal_weights

def create_submission_dict(asset_names, optimal_weights):
    """
    Create the submission dictionary with the required format.
    """
    # Map weights to asset names
    pos_dict = {}
    for i, weight in enumerate(optimal_weights):
        asset_name = asset_names[i]
        pos_dict[asset_name] = weight

    # Submission checker functions
    def get_positions(pos_dict):
        pos = pd.Series(pos_dict)
        pos = pos.replace([np.inf, -np.inf], np.nan)
        pos = pos.dropna()
        pos = pos / pos.abs().sum()
        pos = pos.clip(-0.1, 0.1)
        if pos.abs().max() / pos.abs().sum() > 0.1:
            raise ValueError(f"Portfolio too concentrated {pos.abs().max()=} / {pos.abs().sum()=}")
        return pos

    def get_submission_dict(
        pos_dict,
        your_team_name: str = "moonwalkers",
        your_team_passcode: str = "osihamedmarkmoonwalkers",
    ):
        positions = get_positions(pos_dict)
        # Format the weights to 20 decimal places as strings
        positions = positions.apply(lambda x: f"{x:.20f}")
        return {
            **positions.to_dict(),
            **{
                "team_name": your_team_name,
                "passcode": your_team_passcode,
            },
        }

    # Create the submission dictionary
    submission_dict = get_submission_dict(
        pos_dict,
        your_team_name="moonwalkers",
        your_team_passcode="osihamedmarkmoonwalkers",
    )

    return submission_dict

def submit_to_google_form(submission_string):
    """
    Submit the submission string to the Google Form using Selenium.
    """
    # Define the form URL
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSeUYMkI5ce18RL2aF5C8I7mPxF7haH23VEVz7PQrvz0Do0NrQ/viewform'

    options = webdriver.ChromeOptions()
    options.add_argument(r"--user-data-dir=C:\Users\theha\AppData\Local\Google\Chrome\User Data") #e.g. C:\Users\You\AppData\Local\Google\Chrome\User Data
    options.add_argument(r'--profile-directory=Profile 1') #e.g. Profile 3

    driver = webdriver.Chrome(options=options)

    try:
        # Step 1: Navigate to the Google Form
        driver.get(form_url)
        time.sleep(3)  # Wait for the page to load

        # Step 2: Press the checkbox button
        checkbox = driver.find_element(By.XPATH, '//div[@id="i5"]')
        checkbox.click()
        time.sleep(1)  # Allow time for the checkbox to register

        # Step 3: Fill out the textarea field with your answer
        textarea_container = driver.find_element(By.XPATH, '//div[@class="edhGSc zKHdkd kRy7qc RdH0ib yqQS1"]')
        textarea = textarea_container.find_element(By.XPATH, './/textarea[@aria-label="Your answer"]')
        textarea.send_keys(submission_string)  # Replace with your answer
        time.sleep(2)  # Allow time for input

        # Step 4: Press the Submit button
        submit_button = driver.find_element(By.XPATH, '//span[@class="NPEfkd RveJvd snByac"][text()="Submit"]')
        submit_button.click()
        time.sleep(3)  # Wait for the submission to process

        print("Form submitted successfully.", submission_string)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    last_file_name = None
    while True:
        try:
            # Step 1: Fetch recent messages from the Slack channel
            print("Fetching recent messages...")
            messages = fetch_recent_messages(channel_id)

            if not messages:
                print("No messages found or error in fetching messages.")
                time.sleep(60)  # Wait before retrying
                continue

            # Step 2: Extract file name and passcode
            file_name, passcode = extract_data_file_and_passcode(messages)

            if not (file_name and passcode):
                print("No matching message found or sender did not match.")
                time.sleep(60)  # Wait before retrying
                continue

            # Check if the file name has changed
            if file_name != last_file_name:
                print(f"New data detected. Processing file: {file_name}")

                # Step 3: Authenticate Google Drive
                print("Authenticating with Google Drive...")
                service = authenticate_google_drive()

                # Step 4: Download the file from Google Drive
                print(f"Downloading file: {file_name}")
                if not find_and_download_file(service, file_name):
                    print("File download failed.")
                    time.sleep(60)  # Wait before retrying
                    continue

                # Step 5: Decrypt the file
                print(f"Decrypting file: {file_name}")
                decrypted_df = decrypt_file(file_name, passcode)
                if decrypted_df is None:
                    print("File decryption failed.")
                    time.sleep(60)  # Wait before retrying
                    continue

                # Step 6: Process the data and perform modeling
                asset_names, optimal_weights = process_data(decrypted_df)

                # Step 7: Create submission dictionary
                submission_answer = create_submission_dict(asset_names, optimal_weights)
                print(submission_answer)

                # Convert the dictionary to string
                submission_string = str(submission_answer)
                print(submission_string)

                # Step 8: Submit to Google Form
                submit_to_google_form(submission_string)

                # Update the last_file_name
                last_file_name = file_name

            else:
                print("No new data file detected.")

            # Wait before checking again
            time.sleep(150)  # Check every 60 seconds

        except Exception as e:
            print(f"An error occurred in the monitoring loop: {e}")
            time.sleep(60)  # Wait before retrying


Fetching recent messages...
New data detected. Processing file: release_8475.crypt
Authenticating with Google Drive...
Download 100% complete.
File 'release_8475.crypt' downloaded successfully.
Decrypting file: release_8475.crypt
Decrypted DataFrame:
       strat_0   strat_1   strat_2   strat_3    strat_4    strat_5   strat_6  \
0          NaN       NaN       NaN       NaN        NaN        NaN       NaN   
1    -1.045104       NaN       NaN       NaN        NaN  10.139780  1.481773   
2    -0.229267       NaN       NaN       NaN        NaN  10.109780  0.731916   
3    -0.332475       NaN -0.056052       NaN        NaN   9.961956 -0.557418   
4    -0.827446       NaN -0.349031       NaN        NaN   9.964834 -1.177777   
...        ...       ...       ...       ...        ...        ...       ...   
8471  1.235349 -0.479850  0.201912 -0.023434   9.593660   9.641078 -1.364142   
8472  0.526199 -0.581373 -0.344841 -0.417712  10.209903   9.787503 -0.113452   
8473  0.005310  0.307878 -0.1